In [1]:
import numpy as np
import gym
from gym.wrappers import Monitor
from numpy.random import choice
import random
from phi.api import *
import tensorflow as tf
from tfinterface.reinforcement import DQN, ExpandedStateEnv
import os
from scipy.interpolate import interp1d
import numbers



def get_run():
    try:
        with open("run.txt") as f:
            run = int(f.read().split("/n")[0])
    except:
        run = -1
    
    with open("run.txt", 'w+') as f:
        run += 1
        
        f.seek(0)
        f.write(str(run))
        f.truncate()
        
    return run

In [7]:


run = get_run()
env_logs = '/tmp/cartpole-{}'.format(run)
expansion = 3

env = gym.make('CartPole-v1')
# env = Monitor(env, env_logs)
env = ExpandedStateEnv(env, expansion)
                
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * expansion
model_path = os.getcwd() + "/Q-network-full.model"
logs_path = "logs/run{}".format(run)

[2017-03-03 09:45:16,462] Making new env: CartPole-v1


In [4]:

model = DQN(
    n_actions, n_states,
    model_path = model_path,
    logs_path = logs_path,
    flush_secs = 3.0,
    y = 0.9999,
    buffer_length=500000,
    restore = True
)

print("run: {},\n s: {},\n a: {},\n r: {},\n Qs: {},\n update: {}".format(
    run, model.inputs.s, model.inputs.a, model.inputs.r, model.network.Qs, model.update
))

run: 42,
 s: Tensor("inputs/s:0", shape=(?, 12), dtype=float32, device=/device:CPU:0),
 a: Tensor("inputs/a:0", shape=(?,), dtype=int32, device=/device:CPU:0),
 r: Tensor("inputs/r:0", shape=(?,), dtype=float32, device=/device:CPU:0),
 Qs: Tensor("network/linear_layer/MatMul:0", shape=(?, 2), dtype=float32, device=/device:CPU:0),
 update: name: "network/Adam"
op: "NoOp"
input: "^network/Adam/update_network/relu_layer/kernel/ApplyAdam"
input: "^network/Adam/update_network/linear_layer/kernel/ApplyAdam"
input: "^network/Adam/Assign"
input: "^network/Adam/Assign_1"
device: "/device:CPU:0"



In [5]:
model.fit(
    env, 
    episodes=50000,
    max_episode_length = 60000,
    k = 1000.
)

[MAX] Episode: 0, Length: 68, e: 0.937207122774, learning_rate: 0.937207122774, buffer_len: 68
[NOR] Episode: 0, Length: 68, e: 0.937207122774, learning_rate: 0.937207122774, buffer_len: 68
[NOR] Episode: 10, Length: 27, e: 0.782472613459, learning_rate: 0.782472613459, buffer_len: 279
[NOR] Episode: 20, Length: 14, e: 0.641436818473, learning_rate: 0.641436818473, buffer_len: 560
[NOR] Episode: 30, Length: 45, e: 0.550357732526, learning_rate: 0.550357732526, buffer_len: 818
[NOR] Episode: 40, Length: 41, e: 0.471920717319, learning_rate: 0.471920717319, buffer_len: 1120
[NOR] Episode: 50, Length: 14, e: 0.4095004095, learning_rate: 0.4095004095, buffer_len: 1443
[MAX] Episode: 60, Length: 93, e: 0.354484225452, learning_rate: 0.354484225452, buffer_len: 1822
[NOR] Episode: 60, Length: 93, e: 0.354484225452, learning_rate: 0.354484225452, buffer_len: 1822
[MAX] Episode: 63, Length: 126, e: 0.327546675401, learning_rate: 0.327546675401, buffer_len: 2054
[NOR] Episode: 70, Length: 51, e

KeyboardInterrupt: 

In [8]:
import time

model_run = DQN(
    n_actions, n_states,
    model_path = model_path + ".max",
    flush_secs = 3.0,
    restore = True
)



s = env.reset()
done = False

while not done:
    a = model_run.choose_action(s, e=0.2)
    s, r, done, info = env.step(a)
    env.render()
    time.sleep(0.01)

ArgumentError: argument 2: <type 'exceptions.TypeError'>: wrong type